<a href="https://colab.research.google.com/github/EunjinAn/python_study/blob/main/Finetuning_DK_sentiments_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs
!pip install wandb -q
!pip install -U transformers huggingface_hub -qq
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.6 MB/s eta 0:00:00


In [2]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
from datasets import load_dataset, Dataset, DatasetDict
import torch
from torch import nn
from transformers import Trainer, AdamW, AutoTokenizer, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding
import pandas as pd

In [43]:
# Open Data
df = pd.read_json('https://github.com/aaubs/ds-master/raw/main/data/dk-go-emotions-10k.json.gz')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 26290 to 32230
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  10000 non-null  object
 1   id                    10000 non-null  object
 2   author                10000 non-null  object
 3   subreddit             10000 non-null  object
 4   link_id               10000 non-null  object
 5   parent_id             10000 non-null  object
 6   created_utc           10000 non-null  int64 
 7   rater_id              10000 non-null  int64 
 8   example_very_unclear  10000 non-null  bool  
 9   admiration            10000 non-null  int64 
 10  amusement             10000 non-null  int64 
 11  anger                 10000 non-null  int64 
 12  annoyance             10000 non-null  int64 
 13  approval              10000 non-null  int64 
 14  caring                10000 non-null  int64 
 15  confusion             10000 non-

In [46]:
df.head()

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral,text_dk
26290,Kick him out. Overgrown baby.,eetgbyi,Eminado1,JustNoSO,t3_aiymyl,t3_aiymyl,1548300032,56,False,0,...,0,0,0,0,0,0,0,0,0,"Smid ham ud, forvokset baby."
186025,The internet never disappoints.,edmbo6x,Finito-1994,DunderMifflin,t3_ae1fpa,t1_edlvp9r,1547019136,39,False,0,...,0,0,0,0,0,0,0,0,1,Internettet skuffer aldrig.
3939,We need more vowelled insults so you can spell...,ed3g369,Boydle,bestof,t3_abrkc0,t1_ed38fq9,1546456576,42,False,0,...,0,0,0,0,0,0,0,0,0,"Vi har brug for flere vowelled fornærmelser, s..."
7240,Getting fired would make me leave im sure.,eff34c5,Legion23Golf,Truckers,t3_alfktm,t3_alfktm,1548911872,23,False,0,...,0,0,0,0,0,0,0,0,0,At blive fyret ville få mig til at være sikker.
11688,Growth cannot persist forever in a finite system.,ee6544w,Frogmarsh,changemyview,t3_ag92yz,t1_ee64xkg,1547608704,4,False,0,...,0,0,0,0,0,0,0,0,1,Vækst kan ikke vare ved for evigt i et begræns...


In [45]:
df.columns

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral', 'text_dk'],
      dtype='object')

In [49]:
# Define labels (from column names)
label_cols = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
len(label_cols)

28

In [50]:
# which label has the biggest value?
df['label'] = df[label_cols].idxmax(axis=1)

In [53]:
# DataFrame.sort_values
# DataFrame.sort_index
# Series.sort_values

df['label'].sort_index(ascending = False)

211219           joy
211123       neutral
211103     amusement
211063     confusion
211049    admiration
             ...    
47           neutral
46          approval
31           neutral
30           neutral
4            neutral
Name: label, Length: 10000, dtype: object

In [8]:
dk_labels = ['beundring', 'fornøjelse', 'vrede', 'irritation', 'medhold', 'omsorg', 'forvirring', 'nysgerrighed', 'begær', 'skuffelse', 'misbilligelse', 'afsky', 'forlegenhed', 'spænding', 'frygt', 'taknemmelighed', 'sorg', 'glæde', 'kærlighed', 'nervøsitet', 'optimisme', 'stolthed', 'indsigt', 'lettelse', 'fortrydelse', 'tristhed', 'overraskelse', 'neutral']

In [9]:
dataset = Dataset.from_pandas(df[['text','text_dk','label']])

In [10]:
dataset = dataset.class_encode_column("label")

Casting to class labels:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [54]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text_en', 'text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text_en', 'text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
    valid: Dataset({
        features: ['text_en', 'text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [11]:
# 90% train, 10% test + validation
train_testvalid = dataset.train_test_split(test_size=0.1, stratify_by_column='label')
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, stratify_by_column='label')

# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [12]:
features = dataset['train'].features
features

{'text': Value(dtype='string', id=None),
 'text_dk': Value(dtype='string', id=None),
 'label': ClassLabel(names=['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise'], id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

We need to define label-number-text dictionaries
for both languages (we keep en labels...but dk labels possible)

In [13]:
# english labels, int2str : Conversion integer => class name string.
id2label = {idx:features['label'].int2str(idx) for idx in range(28)}
id2label

{0: 'admiration',
 1: 'amusement',
 2: 'anger',
 3: 'annoyance',
 4: 'approval',
 5: 'caring',
 6: 'confusion',
 7: 'curiosity',
 8: 'desire',
 9: 'disappointment',
 10: 'disapproval',
 11: 'disgust',
 12: 'embarrassment',
 13: 'excitement',
 14: 'fear',
 15: 'gratitude',
 16: 'grief',
 17: 'joy',
 18: 'love',
 19: 'nervousness',
 20: 'neutral',
 21: 'optimism',
 22: 'pride',
 23: 'realization',
 24: 'relief',
 25: 'remorse',
 26: 'sadness',
 27: 'surprise'}

In [14]:
label2id = {v:k for k,v in id2label.items()}
label2id

{'admiration': 0,
 'amusement': 1,
 'anger': 2,
 'annoyance': 3,
 'approval': 4,
 'caring': 5,
 'confusion': 6,
 'curiosity': 7,
 'desire': 8,
 'disappointment': 9,
 'disapproval': 10,
 'disgust': 11,
 'embarrassment': 12,
 'excitement': 13,
 'fear': 14,
 'gratitude': 15,
 'grief': 16,
 'joy': 17,
 'love': 18,
 'nervousness': 19,
 'neutral': 20,
 'optimism': 21,
 'pride': 22,
 'realization': 23,
 'relief': 24,
 'remorse': 25,
 'sadness': 26,
 'surprise': 27}

In [15]:
df['label'].value_counts(normalize=True)

neutral           0.2603
admiration        0.1022
approval          0.0756
annoyance         0.0572
amusement         0.0418
disapproval       0.0409
gratitude         0.0408
curiosity         0.0375
disappointment    0.0360
anger             0.0343
confusion         0.0285
joy               0.0261
optimism          0.0246
love              0.0244
caring            0.0242
realization       0.0234
excitement        0.0198
sadness           0.0190
surprise          0.0163
disgust           0.0144
desire            0.0143
fear              0.0100
remorse           0.0082
embarrassment     0.0066
relief            0.0043
nervousness       0.0042
grief             0.0029
pride             0.0022
Name: label, dtype: float64

In [16]:
checkpoint = 'Maltehb/danish-bert-botxo'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [18]:
dataset['train'][0]['text_dk']

'EU-fascisterne bryder sig ikke om det, når man taler imod det 4. rige.'

In [19]:
t1 = 'Kan Arken fyldes til randen med dyr? Jeg ville [NAME] elske at Dynamite to af enhver art.'

In [20]:
tokenizer(dataset['train'][0]['text_dk'])

{'input_ids': [2, 899, 105, 6866, 5450, 6996, 176, 89, 96, 46, 911, 273, 125, 2230, 1995, 46, 415, 771, 6750, 771, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
# how to set the max_length? max number of the token
def tokenizer_text(examples):
  return tokenizer(examples['text'], truncation=True, max_length=512)

In [22]:
dataset = dataset.map(tokenizer_text, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [23]:
class_weights = (1- (df['label'].value_counts().sort_index() / len(df))).values

In [24]:
class_weights = torch.from_numpy(class_weights).float().to('cuda')

In [70]:
class_weights

tensor([0.8978, 0.9582, 0.9657, 0.9428, 0.9244, 0.9758, 0.9715, 0.9625, 0.9857,
        0.9640, 0.9591, 0.9856, 0.9934, 0.9802, 0.9900, 0.9592, 0.9971, 0.9739,
        0.9756, 0.9958, 0.7397, 0.9754, 0.9978, 0.9766, 0.9957, 0.9918, 0.9810,
        0.9837], device='cuda:0')

In [25]:
class_weights.argmin()

tensor(20, device='cuda:0')

In [71]:
id2label[10]

'disapproval'

In [26]:
id2label[20]

'neutral'

In [27]:
dataset = dataset.rename_columns({'text':'text_en', 'text_dk':'text','label':'labels'})

In [ ]:
dataset['train'][0]

In [74]:
class WeightedLossTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    # Feed inputs to model and extract logits
    outputs = model(**inputs)
    logits = outputs.get('logits')
    # Extract labels
    labels = inputs.get('labels')
    # Define loss function with class weights
    loss_func = nn.CrossEntropyLoss(weight=class_weights)
    # Compute loss
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

In [30]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=28,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at Maltehb/danish-bert-botxo were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [76]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  return {"f1": f1}


In [32]:
from transformers import TrainingArguments

batch_size = 32
logging_steps = len(dataset['train']) // batch_size
output_dir = 'dk_emotion_bert_in_class'

training_args = TrainingArguments(output_dir=output_dir,
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy='epoch',
                                  logging_steps=logging_steps,
                                  fp16=True,
                                  report_to="wandb",
                                  run_name="bert-dk-emotion-sloppy-start-in-class",
                                  push_to_hub=True
                                  )


In [33]:
%env WANDB_PROJECT=dk_sentiment_transformers

env: WANDB_PROJECT=dk_sentiment_transformers


In [34]:
from huggingface_hub import notebook_login

In [38]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [39]:
trainer = WeightedLossTrainer(model = model,
                              args = training_args,
                              compute_metrics = compute_metrics,
                              train_dataset = dataset["train"],
                              eval_dataset = dataset["valid"],
                              tokenizer = tokenizer
                              )

Cloning https://huggingface.co/EJaalborg2022/dk_emotion_bert_in_class into local empty directory.
Using cuda_amp half precision backend


In [40]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, text_en. If __index_level_0__, text, text_en are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9000
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1410
  Number of trainable parameters = 110638876
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLE

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,2.845600,2.638545,0.192578
2,2.580900,2.485080,0.236232
3,2.422100,2.422077,0.258940
4,2.286400,2.388591,0.286355
5,2.203800,2.375027,0.296639


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text, text_en. If __index_level_0__, text, text_en are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to dk_emotion_bert_in_class/checkpoint-500
Configuration saved in dk_emotion_bert_in_class/checkpoint-500/config.json
Model weights saved in dk_emotion_bert_in_class/checkpoint-500/pytorch_model.bin
tokenizer config file saved in dk_emotion_bert_in_class/checkpoint-500/tokenizer_config.json
Special tokens file saved in dk_emotion_bert_in_class/checkpoint-500/special_tokens_map.json
tokenizer config file saved in dk_emotion_bert_in_class/tokenizer_config.json
Special tokens file saved in dk_emotion_bert_in_class/special_tokens_map.json
The following columns in the evaluation set don't h

TrainOutput(global_step=1410, training_loss=2.4663146810328707, metrics={'train_runtime': 275.6475, 'train_samples_per_second': 163.252, 'train_steps_per_second': 5.115, 'total_flos': 1104650023625472.0, 'train_loss': 2.4663146810328707, 'epoch': 5.0})

In [55]:
trainer.push_to_hub(output_dir)

Saving model checkpoint to dk_emotion_bert_in_class
Configuration saved in dk_emotion_bert_in_class/config.json
Model weights saved in dk_emotion_bert_in_class/pytorch_model.bin
tokenizer config file saved in dk_emotion_bert_in_class/tokenizer_config.json
Special tokens file saved in dk_emotion_bert_in_class/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/422M [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/EJaalborg2022/dk_emotion_bert_in_class
   105d940..004e108  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/EJaalborg2022/dk_emotion_bert_in_class
   105d940..004e108  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.2966390941425724}]}
To https://huggingface.co/EJaalborg2022/dk_emotion_bert_in_class
   004e108..7c4a09f  main -> main

   004e108..7c4a09f  main -> main



'https://huggingface.co/EJaalborg2022/dk_emotion_bert_in_class/commit/004e108097927e7212dae7b60cead6f50ad31777'

In [56]:
model.push_to_hub(output_dir)

Configuration saved in dk_emotion_bert_in_class/config.json
Model weights saved in dk_emotion_bert_in_class/pytorch_model.bin
Uploading the following files to EJaalborg2022/dk_emotion_bert_in_class: pytorch_model.bin,config.json


CommitInfo(commit_url='https://huggingface.co/EJaalborg2022/dk_emotion_bert_in_class/commit/975a9f12714721e627752d8904ad257ca288ef8c', commit_message='Upload BertForSequenceClassification', commit_description='', oid='975a9f12714721e627752d8904ad257ca288ef8c', pr_url=None, pr_revision=None, pr_num=None)

In [57]:
dataset.push_to_hub('go-emotion-dk-autotranlated-10k')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
from transformers import pipeline

model_ckpt = 'RJuro/dk_emotion_bert_in_class'
pipe = pipeline('text-classification', model=model_ckpt)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--RJuro--dk_emotion_bert_in_class/snapshots/35f5efafcbd99f772b7cf1c90b66c3afe97802dc/config.json
Model config BertConfig {
  "_name_or_path": "RJuro/dk_emotion_bert_in_class",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "admiration",
    "1": "amusement",
    "2": "anger",
    "3": "annoyance",
    "4": "approval",
    "5": "caring",
    "6": "confusion",
    "7": "curiosity",
    "8": "desire",
    "9": "disappointment",
    "10": "disapproval",
    "11": "disgust",
    "12": "embarrassment",
    "13": "excitement",
    "14": "fear",
    "15": "gratitude",
    "16": "grief",
    "17": "joy",
    "18": "love",
    "19": "nervousness",
    "20": "neutral

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--RJuro--dk_emotion_bert_in_class/snapshots/35f5efafcbd99f772b7cf1c90b66c3afe97802dc/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at RJuro/dk_emotion_bert_in_class.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--RJuro--dk_emotion_bert_in_class/snapshots/35f5efafcbd99f772b7cf1c90b66c3afe97802dc/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--RJuro--dk_emotion_bert_in_class/snapshots/35f5efafcbd99f772b7cf1c90b66c3afe97802dc/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--RJuro--dk_emotion_bert_in_class/snapshots/35f5efafcbd99f772b7cf1c90b66c3afe97802dc/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--RJuro--dk_emotion_bert_in_class/snapshots/35f5efafcbd99f772b7cf1c90b66c3afe97802dc/tokenizer_config.json


In [59]:
pipe(['Du er en idiot!', 'Jeg er sulten!'])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'anger', 'score': 0.16308538615703583},
 {'label': 'admiration', 'score': 0.20328542590141296}]